In [1]:
'''
Unify Encoding:

'*': light bulb  
'#': wall(black square) without number  
'0-4': wall(black square) with number  
'A-Z': wormhole (same character on different grid means same pair of wormholes)
'_': empty cell(no wall/light bulb/wormhole)
'!': can not put light bulb
'^': empty but lit by other bulbs
'''

"\nUnify Encoding:\n\n'*': light bulb  \n'#': wall(black square) without number  \n'0-4': wall(black square) with number  \n'A-Z': wormhole (same character on different grid means same pair of wormholes)\n'_': empty cell(no wall/light bulb/wormhole)\n'!': can not put light bulb\n'^': empty but lit by other bulbs\n"

In [2]:
import copy
import random
import time 
from tkinter import _flatten

# Creating Puzzle

## Step1: Initialization

In [3]:
def init_grids(size:tuple):
    '''
    Initialize grids based on given size.
    parameter:
        size: size of the grids(eg:(4,4) both grids are 4 by 4 grid)
    return: 
        grids: [grid0, grid1]
        empty_loc: emptu locations(all)
    '''
    grid0 = []
    tp_empty_loc0 = []
    
    for i in range(size[0]):
        grid0.append([])
        for j in range(size[1]):
            grid0[i].append('_')
            tp_empty_loc0.append((i,j))
            
    grid1 = copy.deepcopy(grid0)
    tp_empty_loc1 = copy.deepcopy(tp_empty_loc0)
    
    init_grids = [grid0,grid1]
    empty_loc = [tp_empty_loc0, tp_empty_loc1]
    
    return init_grids, empty_loc

## Step2: Add Walls

In [4]:
def add_walls(init_grids: list, wall_per: float):
    '''
    Add walls on init_grid, based on wall_per.
    parameter：
        init_grid: empty initialized grids
        wall_per: percentage of black squares
    return:
        grids: grids after adding walls
        n_walls: number of walls in each grid
        walls_loc: walls' location
    '''
    grids = init_grids
    r, c = len(init_grids[0]), len(init_grids[0][0])
    n_cells = r*c
    n_walls = int(n_cells*wall_per)
    walls_loc = []
    for i in range(2):
        tp = random.sample(range(n_cells), n_walls)
        walls_loc.append([])
        for j in tp:
            tp_i = j//c
            tp_j = j%c
            grids[i][tp_i][tp_j] = '#'
            walls_loc[i].append((tp_i,tp_j))
    return grids,n_walls,walls_loc

## Step3: Add Wormholes 

In [5]:
def add_wormholes(walls_added_grids:list, n_wormholes:int, empty_loc:list):
    '''
    Add wormholes on grids that already add walls
    parameter:
        walls_added_grids: grids already add walls
        n_wormholes: number of wormholes in each grid
        empty_loc: list of locations that are still empty
    return:
        wormhole_added_grids: grids already add wormholes
        empty_loc: list of locations that are still empty
    '''
    wormhole_names = [chr(i) for i in range(65,91)]
    for n in range(n_wormholes):
        (tp_0_i, tp_0_j) = random.sample(empty_loc[0], 1)[0]
        walls_added_grids[0][tp_0_i][tp_0_j] = wormhole_names[n]
        empty_loc[0].remove((tp_0_i, tp_0_j))
        (tp_1_i, tp_1_j) = random.sample(empty_loc[1], 1)[0]
        empty_loc[1].remove((tp_1_i, tp_1_j))
        walls_added_grids[1][tp_1_i][tp_1_j] = wormhole_names[n]
    wormhole_added_grids = walls_added_grids
    return wormhole_added_grids, empty_loc

## Step4: Add Light Bulbs

In [6]:
test_grid,empty_loc = init_grids((6,6))
test_grid, n_walls, walls_loc = add_walls(test_grid,0.25)
test_grid
for i in range(2):
    for j in walls_loc[i]:
        empty_loc[i].remove(j)

wormhole_added_grids, empty_loc = add_wormholes(test_grid, 2, empty_loc)

empty_and_not_lit_loc = copy.deepcopy(empty_loc)


In [7]:
# create prio_bulb_loc(the cells adjacent to walls)
prio_bulb_loc = []
for i in range(len(empty_loc)): # i 第几个grid eg:0
    prio_bulb_loc.append([])
    for j in empty_loc[i]: # j 当前这个grid的某个empty loc eg:(0,2)
        flag = False
        if j[0]-1>=0: # up
            if wormhole_added_grids[i][j[0]-1][j[1]] == '#':
                flag = True
        if j[0]+1<len(wormhole_added_grids[i]): # down
            if wormhole_added_grids[i][j[0]+1][j[1]] == '#':
                flag = True
        if j[1]-1>=0: #left
            if wormhole_added_grids[i][j[0]][j[1]-1] == '#':
                flag = True
        if j[1]+1<len(wormhole_added_grids[i][0]): # right
            if wormhole_added_grids[i][j[0]][j[1]+1] == '#':
                flag = True
        if flag:
            prio_bulb_loc[i].append(j)

prio_bulb_loc

[[(0, 1),
  (0, 4),
  (1, 0),
  (1, 2),
  (1, 4),
  (1, 5),
  (2, 2),
  (2, 4),
  (3, 1),
  (3, 3),
  (3, 5),
  (4, 0),
  (4, 2),
  (4, 4),
  (5, 1),
  (5, 3),
  (5, 4)],
 [(0, 0),
  (0, 2),
  (0, 4),
  (1, 0),
  (1, 1),
  (1, 5),
  (2, 1),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 5),
  (4, 1),
  (4, 4),
  (4, 5),
  (5, 0),
  (5, 2),
  (5, 3),
  (5, 4)]]

In [8]:
def light_up(wormhole_added_grids: list, bulb_loc: list, empty_and_not_lit_loc: list, prio_bulb_loc: list):
    '''
    update record after adding a bulb on bulb_loc
    parameters:
        wormhole_added_grids: grids already add wormholes
        bulb_loc: the location that you want to put a bulb 
            eg:[x,y,z] - xth gird, yth row, zth column
        empty_and_not_lit_loc: location of empty and not lit cells
    return:
        wormhole_added_grids: grids after adding a bulb on bulb_loc
        empty_and_not_lit_loc: location of empty and not lit cells
    '''
    wormhole_added_grids[bulb_loc[0]][bulb_loc[1]][bulb_loc[2]] = 'b'
    # light up
    
    ## up
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[1] -= 1
    while cur_loc[1]>=0:
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            if (cur_loc[1], cur_loc[2]) in prio_bulb_loc[cur_loc[0]]:
                prio_bulb_loc[cur_loc[0]].remove((cur_loc[1], cur_loc[2]))
            
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#' or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
            break
        # case4: jump in a loop of wormhole
        if cur_loc == bulb_loc:
            break
            
        cur_loc[1] -= 1
        
    ## down
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[1] += 1
    while cur_loc[1]<len(wormhole_added_grids[cur_loc[0]]):
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            if (cur_loc[1], cur_loc[2]) in prio_bulb_loc[cur_loc[0]]:
                prio_bulb_loc[cur_loc[0]].remove((cur_loc[1], cur_loc[2]))
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#' or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
            break    
        # case4: jump in a loop of wormhole
        if cur_loc == bulb_loc:
            break
            
        cur_loc[1] += 1
        
    ## left
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[2] -= 1
    while cur_loc[2]>=0:
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            if (cur_loc[1], cur_loc[2]) in prio_bulb_loc[cur_loc[0]]:
                prio_bulb_loc[cur_loc[0]].remove((cur_loc[1], cur_loc[2]))
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#'or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
            break
        # case4: jump in a loop of wormhole
        if cur_loc == bulb_loc:
            break
            
        cur_loc[2] -= 1
        
    ## right
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[2] += 1
    while cur_loc[2]<len(wormhole_added_grids[cur_loc[0]][0]):
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            if (cur_loc[1], cur_loc[2]) in prio_bulb_loc[cur_loc[0]]:
                prio_bulb_loc[cur_loc[0]].remove((cur_loc[1], cur_loc[2]))
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#' or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
        # case4: jump in a loop of wormhole
            break
        if cur_loc == bulb_loc:
            break
            
        cur_loc[2] += 1
        
    return wormhole_added_grids, empty_and_not_lit_loc, prio_bulb_loc

In [9]:
# test light_up function
# light_up(wormhole_added_grids, [0,0,1], empty_and_not_lit_loc)

In [10]:
# test wormhole loop case
test_grid = [[['_', 'A', '_'],['_', '_', '_'],['_', 'B', '_']],
             [['_', 'B', '_'],['_', '_', '_'],['_', 'A', '_']]]
light_up(test_grid, [0,1,1], [[(0,0),(0,2),(1,0),(1,1),(1,2),(2,0),(2,2)],[(0,0),(0,2),(1,0),(1,1),(1,2),(2,0),(2,2)]],[[],[]])

([[['_', 'A', '_'], ['^', 'b', '^'], ['_', 'B', '_']],
  [['_', 'B', '_'], ['_', '^', '_'], ['_', 'A', '_']]],
 [[(0, 0), (0, 2), (1, 1), (2, 0), (2, 2)],
  [(0, 0), (0, 2), (1, 0), (1, 2), (2, 0), (2, 2)]],
 [[], []])

In [11]:
prio_bulb_loc

[[(0, 1),
  (0, 4),
  (1, 0),
  (1, 2),
  (1, 4),
  (1, 5),
  (2, 2),
  (2, 4),
  (3, 1),
  (3, 3),
  (3, 5),
  (4, 0),
  (4, 2),
  (4, 4),
  (5, 1),
  (5, 3),
  (5, 4)],
 [(0, 0),
  (0, 2),
  (0, 4),
  (1, 0),
  (1, 1),
  (1, 5),
  (2, 1),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 5),
  (4, 1),
  (4, 4),
  (4, 5),
  (5, 0),
  (5, 2),
  (5, 3),
  (5, 4)]]

In [12]:
wormhole_added_grids

[[['#', '_', '_', 'A', '_', '#'],
  ['_', '_', '_', '#', '_', '_'],
  ['_', '_', '_', '#', '_', '_'],
  ['_', '_', '_', '_', '#', '_'],
  ['_', '#', '_', '#', '_', '#'],
  ['B', '_', '_', '_', '_', '#']],
 [['_', '#', '_', '_', '_', '#'],
  ['_', '_', '_', '_', '_', '_'],
  ['#', '_', '_', 'B', '_', '#'],
  ['_', '_', '_', 'A', '_', '_'],
  ['#', '_', '#', '#', '_', '_'],
  ['_', '#', '_', '_', '_', '#']]]

In [13]:
def add_light_bulbs(wormhole_added_grids:list, empty_and_not_lit_loc:list, empty_loc:list ,prio_bulb_loc:list):
    '''
    Add light bulbs on girds until all cells are not empty or lit by other bulbs. To make it much easier to use numbers on walls 
    to restrict bulbs' location(higher probability to be unique solution), add bulbs in prio_bulb_loc first randomly, 
    and then consider other location. 
    parameters:
        wormhole_added_grids: grids already add wormholes
        empty_and_not_lit_loc: location of empty and not lit cells
        empty_loc: list of locations that are still empty
        prio_bulb_loc: the cells adjacent to walls
    return:
        bulb_added_grids: grids after adding light bulbs
    '''
    flag = True
    #while len(empty_and_not_lit_loc)>0:
    while flag:
        
        # random select a bulb_loc
        ## random select a grid
        if len(empty_and_not_lit_loc[0])>0 and len(empty_and_not_lit_loc[1])>0:
            tp_grid_index = random.sample(range(len(wormhole_added_grids)),1)[0]
        elif len(empty_and_not_lit_loc[0])==0:
            tp_grid_index = 1
        elif len(empty_and_not_lit_loc[1])==0:
            tp_grid_index = 0
        ## random select a cell in current grid
        if len(prio_bulb_loc[tp_grid_index])>0:
            tp_loc = random.sample(prio_bulb_loc[tp_grid_index],1)[0]
            prio_bulb_loc[tp_grid_index].remove(tp_loc)
            empty_and_not_lit_loc[tp_grid_index].remove(tp_loc)
            empty_loc[tp_grid_index].remove(tp_loc)
        elif len(prio_bulb_loc[tp_grid_index])==0:
            tp_loc = random.sample(empty_and_not_lit_loc[tp_grid_index],1)[0]
            empty_and_not_lit_loc[tp_grid_index].remove(tp_loc)
            empty_loc[tp_grid_index].remove(tp_loc)        
        wormhole_added_grids, empty_and_not_lit_loc, prio_bulb_loc = light_up(wormhole_added_grids, [tp_grid_index,tp_loc[0],tp_loc[1]], empty_and_not_lit_loc, prio_bulb_loc)
        
        # if no cell is available:
        if len(empty_and_not_lit_loc[0])==0 and len(empty_and_not_lit_loc[1])==0:
            flag = False
            
        bulb_added_grids = wormhole_added_grids
    return bulb_added_grids

In [14]:
bulb_added_grids = add_light_bulbs(wormhole_added_grids, empty_and_not_lit_loc, empty_loc ,prio_bulb_loc)
display(bulb_added_grids)

[[['#', 'b', '^', 'A', '^', '#'],
  ['^', '^', '^', '#', 'b', '^'],
  ['^', '^', 'b', '#', '^', '^'],
  ['^', '^', '^', 'b', '#', 'b'],
  ['b', '#', '^', '#', 'b', '#'],
  ['B', '^', '^', '^', '^', '#']],
 [['b', '#', 'b', '^', '^', '#'],
  ['^', '^', '^', '^', '^', 'b'],
  ['#', 'b', '^', 'B', 'b', '#'],
  ['b', '^', '^', 'A', '^', '^'],
  ['#', '^', '#', '#', '^', 'b'],
  ['b', '#', '^', 'b', '^', '#']]]

## Step5: Add numbers on walls

In [15]:
walls_loc

[[(0, 0), (1, 3), (4, 3), (4, 5), (4, 1), (0, 5), (3, 4), (5, 5), (2, 3)],
 [(0, 1), (4, 0), (2, 0), (5, 1), (0, 5), (2, 5), (4, 2), (4, 3), (5, 5)]]

In [16]:
prio_bulb_loc

[[], []]

In [17]:
random.sample([(1, 3), (1, 0), (0, 0), (3, 2)],1)

[(1, 3)]

In [18]:
def add_numbers_on_walls(bulb_added_grids: list, walls_loc:list, hard:bool):
    '''
    Add numbers on walls to restrict bulb location, based on difficult:
        easy: record numbers on all walls
        hard: remain some walls without bulb number
    parameters:
        bulb_added_grids: grids after adding light bulbs
        walls_loc: walls' location
        hard: difficulty(True: hard, False: easy)
    return:
        
    '''
    hard_ind = 0.8 # update around 80% walls
    update_walls_loc = copy.deepcopy(walls_loc)
    if hard:
        for i in range(2):
            del_index = copy.deepcopy(random.sample(walls_loc[i],round(len(walls_loc[i])*(1-hard_ind))))
            for j in del_index:
                update_walls_loc[i].remove(j)
                
    numbers_on_walls = []
    for i in range(2):
        #numbers_on_walls.append([])
        for j in update_walls_loc[i]:
            tp = 0
            if j[0]-1>=0: # up
                if bulb_added_grids[i][j[0]-1][j[1]] == 'b':
                    tp += 1
            if j[0]+1<len(bulb_added_grids[i]): # down
                if bulb_added_grids[i][j[0]+1][j[1]] == 'b':
                    tp += 1
            if j[1]-1>=0: #left
                if bulb_added_grids[i][j[0]][j[1]-1] == 'b':
                    tp += 1
            if j[1]+1<len(bulb_added_grids[i][0]): # right
                if bulb_added_grids[i][j[0]][j[1]+1] == 'b':
                    tp += 1
            #numbers_on_walls[i].append(tp)
            bulb_added_grids[i][j[0]][j[1]] = tp
    number_added_grids = bulb_added_grids
    return number_added_grids

In [19]:
number_added_grids = add_numbers_on_walls(bulb_added_grids, walls_loc, True)
display(number_added_grids)

[[[1, 'b', '^', 'A', '^', 0],
  ['^', '^', '^', 1, 'b', '^'],
  ['^', '^', 'b', '#', '^', '^'],
  ['^', '^', '^', 'b', 3, 'b'],
  ['b', 1, '^', 2, 'b', 2],
  ['B', '^', '^', '^', '^', '#']],
 [['b', 2, 'b', '^', '^', '#'],
  ['^', '^', '^', '^', '^', 'b'],
  [2, 'b', '^', 'B', 'b', 2],
  ['b', '^', '^', 'A', '^', '^'],
  [2, '^', 0, 1, '^', 'b'],
  ['b', 1, '^', 'b', '^', '#']]]

In [20]:
# create an unverified puzzle:
for g in range(len(number_added_grids)):
    for i in range(len(number_added_grids[g])):
        for j in range(len(number_added_grids[g][i])):
            if number_added_grids[g][i][j] == 'b' or number_added_grids[g][i][j] == '^':
                number_added_grids[g][i][j] = '_'

In [21]:
display(number_added_grids)

[[[1, '_', '_', 'A', '_', 0],
  ['_', '_', '_', 1, '_', '_'],
  ['_', '_', '_', '#', '_', '_'],
  ['_', '_', '_', '_', 3, '_'],
  ['_', 1, '_', 2, '_', 2],
  ['B', '_', '_', '_', '_', '#']],
 [['_', 2, '_', '_', '_', '#'],
  ['_', '_', '_', '_', '_', '_'],
  [2, '_', '_', 'B', '_', 2],
  ['_', '_', '_', 'A', '_', '_'],
  [2, '_', 0, 1, '_', '_'],
  ['_', 1, '_', '_', '_', '#']]]

In [22]:
def create_unverified_puzzle(grid_size: tuple, wall_per: float, hard:bool):
    '''
    Create unverified puzzle
    parameters:
        grid_size: size of the grids(eg:(4,4) both grids are 4 by 4 grid)
        wall_per: percentage of black squares
        eg:grid_size = (6,6), wall_per = 0.25, hard = True
    '''
    #test_grid,empty_loc = init_grids((6,6))
    test_grid,empty_loc = init_grids(grid_size)
    #test_grid, n_walls, walls_loc = add_walls(test_grid,0.25)
    test_grid, n_walls, walls_loc = add_walls(test_grid,wall_per)
    for i in range(2):
        for j in walls_loc[i]:
            empty_loc[i].remove(j)
    wormhole_added_grids, empty_loc = add_wormholes(test_grid, 2, empty_loc)
    empty_and_not_lit_loc = copy.deepcopy(empty_loc)
    prio_bulb_loc = []
    #cal prio_bulb_loc
    for i in range(len(empty_loc)): # i 第几个grid eg:0
        prio_bulb_loc.append([])
        for j in empty_loc[i]: # j 当前这个grid的某个empty loc eg:(0,2)
            flag = False
            if j[0]-1>=0: # up
                if wormhole_added_grids[i][j[0]-1][j[1]] == '#':
                    flag = True
            if j[0]+1<len(wormhole_added_grids[i]): # down
                if wormhole_added_grids[i][j[0]+1][j[1]] == '#':
                    flag = True
            if j[1]-1>=0: #left
                if wormhole_added_grids[i][j[0]][j[1]-1] == '#':
                    flag = True
            if j[1]+1<len(wormhole_added_grids[i][0]): # right
                if wormhole_added_grids[i][j[0]][j[1]+1] == '#':
                    flag = True
            if flag:
                prio_bulb_loc[i].append(j)    
    bulb_added_grids = add_light_bulbs(wormhole_added_grids, empty_and_not_lit_loc, empty_loc ,prio_bulb_loc)
    number_added_grids = add_numbers_on_walls(bulb_added_grids, walls_loc, hard)
    #number_added_grids = add_numbers_on_walls(bulb_added_grids, walls_loc, True)
    # create an unverified puzzle:
    for g in range(len(number_added_grids)):
        for i in range(len(number_added_grids[g])):
            for j in range(len(number_added_grids[g][i])):
                if number_added_grids[g][i][j] == 'b' or number_added_grids[g][i][j] == '^':
                    number_added_grids[g][i][j] = '_'
    return number_added_grids

## Step6: Verify whether it is unique solution

In [23]:
############# TO Do: Use solving function to verify ##########

# Solving Puzzle

In [24]:
'''
Unify Encoding:

'b': light bulb  
'#': wall(black square) without number  
'0-4': wall(black square) with number  
'A-Z': wormhole (same character on different grid means same pair of wormholes)
'_': empty cell(no wall/light bulb/wormhole)
'!': can not put light bulb
'^': empty but lit by other bulbs
'''

"\nUnify Encoding:\n\n'b': light bulb  \n'#': wall(black square) without number  \n'0-4': wall(black square) with number  \n'A-Z': wormhole (same character on different grid means same pair of wormholes)\n'_': empty cell(no wall/light bulb/wormhole)\n'!': can not put light bulb\n'^': empty but lit by other bulbs\n"

## To Saxue:
### TO DO: 
#### 1. Solving puzzle part (logic can refer to the pdf,  deduction first, if not enough, dfs)
#### 2. Display

In [25]:
number_added_grids

[[[1, '_', '_', 'A', '_', 0],
  ['_', '_', '_', 1, '_', '_'],
  ['_', '_', '_', '#', '_', '_'],
  ['_', '_', '_', '_', 3, '_'],
  ['_', 1, '_', 2, '_', 2],
  ['B', '_', '_', '_', '_', '#']],
 [['_', 2, '_', '_', '_', '#'],
  ['_', '_', '_', '_', '_', '_'],
  [2, '_', '_', 'B', '_', 2],
  ['_', '_', '_', 'A', '_', '_'],
  [2, '_', 0, 1, '_', '_'],
  ['_', 1, '_', '_', '_', '#']]]

In [26]:
walls_loc


[[(0, 0), (1, 3), (4, 3), (4, 5), (4, 1), (0, 5), (3, 4), (5, 5), (2, 3)],
 [(0, 1), (4, 0), (2, 0), (5, 1), (0, 5), (2, 5), (4, 2), (4, 3), (5, 5)]]

In [27]:
number_added_grids

[[[1, '_', '_', 'A', '_', 0],
  ['_', '_', '_', 1, '_', '_'],
  ['_', '_', '_', '#', '_', '_'],
  ['_', '_', '_', '_', 3, '_'],
  ['_', 1, '_', 2, '_', 2],
  ['B', '_', '_', '_', '_', '#']],
 [['_', 2, '_', '_', '_', '#'],
  ['_', '_', '_', '_', '_', '_'],
  [2, '_', '_', 'B', '_', 2],
  ['_', '_', '_', 'A', '_', '_'],
  [2, '_', 0, 1, '_', '_'],
  ['_', 1, '_', '_', '_', '#']]]

In [28]:


def count_neb(i,j,puzzle,index,sign):
    count=0
    if i-1>=0 and puzzle[index][i-1][j]==sign :
        count+=1
    if j-1>=0 and puzzle[index][i][j-1]==sign:
        count+=1
    if i+1<len(puzzle[0]) and puzzle[index][i+1][j]==sign:
        count+=1
    if j+1<len(puzzle[0][0]) and puzzle[index][i][j+1]==sign :
        count+=1
    return count
        
        
def place_bulbs(walls_loc,puzzle):
    index=0
    b_list=[]
    for i in range(len(walls_loc)):
        if(i==1):
            index=1
        for j in walls_loc[i]:  
            count1=count_neb(j[0],j[1],puzzle,index,"b")
            count2=count_neb(j[0],j[1],puzzle,index,"_")
            if(type(puzzle[index][j[0]][j[1]])==int and puzzle[index][j[0]][j[1]]>=count1+count2):
                
                place_bulbs_step1(puzzle,j[0],j[1],index,"b", b_list)            
         
    return b_list

def place_bulbs_step1(puzzle,i,j,index,sign, b_list):
    if i-1>=0 and puzzle[index][i-1][j]=="_" :
        puzzle[index][i-1][j]=sign
        b_list.append([index,i-1,j])
    if j-1>=0 and puzzle[index][i][j-1]=="_":
        puzzle[index][i][j-1]=sign
        b_list.append([index,i,j-1])
    if i+1<len(puzzle[0]) and puzzle[index][i+1][j]=="_":
        puzzle[index][i+1][j]=sign
        b_list.append([index,i+1,j])
    if j+1<len(puzzle[0][0]) and puzzle[index][i][j+1]=="_":
        puzzle[index][i][j+1]=sign
        b_list.append([index,i,j+1])
        

def get_no_bulb_list(puzzle,i,j,index, b_list):
    if i-1>=0 and puzzle[index][i-1][j]=="_" :
        b_list.append([index,i-1,j])
    if j-1>=0 and puzzle[index][i][j-1]=="_":
    
        b_list.append([index,i,j-1])
    if i+1<len(puzzle[0]) and puzzle[index][i+1][j]=="_":
       
        b_list.append([index,i+1,j])
    if j+1<len(puzzle[0][0]) and puzzle[index][i][j+1]=="_":
       
        b_list.append([index,i,j+1])
    return b_list
def no_bulbs(walls_loc,puzzle):
    index=0
    test=[]
    for i in range(len(walls_loc)):
        if(i==1):
            index=1
        for j in walls_loc[i]:
            count=count_neb(j[0],j[1],puzzle,index,"b")
            if puzzle[index][j[0]][j[1]]==0 or (type(puzzle[index][j[0]][j[1]])==int and puzzle[index][j[0]][j[1]]<=count):
                #print("no bubls around")
                get_no_bulb_list(puzzle,j[0],j[1],index,test)
                #res.append([index,j[0],j[1]])
    return test
     


In [29]:
def generate_empty(puzzle):
    empty_loc=[]
    for index in range(len(puzzle)):
        sub=[]
        for i in range(len(puzzle[index])):
            for j in range(len(puzzle[index][0])):
                if(puzzle[index][i][j]=="_"):
                     sub.append((i,j))
        empty_loc.append(sub)
    return empty_loc



In [30]:
# create prio_bulb_loc(the cells adjacent to walls)
def generate_wall_neb(empty_loc,puzzle):
    prio_bulb_loc = []
    for i in range(len(empty_loc)): # 
        prio_bulb_loc.append([])
        for j in empty_loc[i]: #
            flag = False
            if j[0]-1>=0: # up
                if puzzle[i][j[0]-1][j[1]] == '#':
                    flag = True
            if j[0]+1<len(puzzle[i]): # down
                if puzzle[i][j[0]+1][j[1]] == '#':
                    flag = True
            if j[1]-1>=0: #left
                if puzzle[i][j[0]][j[1]-1] == '#':
                    flag = True
            if j[1]+1<len(puzzle[i][0]): # right
                if puzzle[i][j[0]][j[1]+1] == '#':
                    flag = True
            if flag:
                prio_bulb_loc[i].append(j)

    return prio_bulb_loc


In [31]:
def light_step1(curr_bulbs,puzzle,empty_loc, prio_bulb_loc):
    for bulb in curr_bulbs:
        light_up(puzzle, bulb, empty_loc, prio_bulb_loc)

In [32]:
def find_empty(puzzle):
    not_lit_loc=[]
    for index in range(len(puzzle)):
        sub=[]
        for i in range(len(puzzle[index])):
            for j in range(len(puzzle[index][0])):
                if(puzzle[index][i][j]=="_" ):
                    sub.append((i,j))
        not_lit_loc.append(sub)
    return not_lit_loc

def lit_position(puzzle):
    res=[]
    for index in range(len(puzzle)):
        sub=[]
        for i in range(len(puzzle[index])):
            for j in range(len(puzzle[index][0])):
                if(puzzle[index][i][j]=="^"):
                    sub.append((i,j))
        res.append(sub)
    return res


In [33]:
#display(puzzle)

In [34]:
def is_valid(puzzle,i,j,index):
    #print("start to check",puzzle[index][i][j],[index,i,j])
    if(i-1>=0):
        if(type(puzzle[index][i-1][j])==int):
            bulb_around=count_neb(i-1,j,puzzle,index,"b")
            #print( bulb_around,puzzle[index][i-1][j],"bulb1")
            if(puzzle[index][i-1][j]<=bulb_around):
                return False
    if(i+1<len(puzzle[index])):
        if(type(puzzle[index][i+1][j])==int):
            bulb_around=count_neb(i+1,j,puzzle,index,"b")
            #print( bulb_around,puzzle[index][i+1][j],"bulb2")
            if(puzzle[index][i+1][j]<=bulb_around):
                return False
    if(j-1>=0):
        if(type(puzzle[index][i][j-1])==int):
            bulb_around=count_neb(i,j-1,puzzle,index,"b")
            #print( bulb_around,puzzle[index][i][j-1],"bulb3")
            if(puzzle[index][i][j-1]<=bulb_around):
                return False
    if(j+1<len(puzzle[index][0])):
        if(type(puzzle[index][i][j+1])==int):
            bulb_around=count_neb(i,j+1,puzzle,index,"b")
            #print( bulb_around,puzzle[index][i][j-1],"bulb4")
            if(puzzle[index][i][j+1]<=bulb_around):
                return False
    return True
    
    
    

def check(puzzle,cell):
    row=cell[1]
    col=cell[2]

    #print("------------------------1")
    while row>0:
        #print("------------------------1")
        row=row-1
        #print(cell,puzzle[cell[0]][row][col])
        if(puzzle[cell[0]][row][col]=="b"):
            #print(cell, "no pass1")
            return False
        if(puzzle[cell[0]][row][col]=="#" or str(puzzle[cell[0]][row][col]).isupper() or type(puzzle[cell[0]][row][col])==int):
            break
    row=cell[1]
    while row<len(puzzle[cell[0]])-1:
        #print("------------------------2")
        row=row+1
        if(puzzle[cell[0]][row][col]=="b"):
            #print(puzzle[cell[0]][row][col],cell, "no pass2")
            return False
        if(puzzle[cell[0]][row][col]=="#"or str(puzzle[cell[0]][row][col]).isupper() or type(puzzle[cell[0]][row][col])==int):
            break
 
    row=cell[1]
    while col<len(puzzle[cell[0]][0])-1:
        #print("------------------------3")
        col=col+1

        if(puzzle[cell[0]][row][col]=="b"):
            #print(cell,puzzle[cell[0]][row][col], "no pass3")
            return False
        if(puzzle[cell[0]][row][col]=="#" or str(puzzle[cell[0]][row][col]).isupper() or type(puzzle[cell[0]][row][col])==int):
            break
    col=cell[2]
    while col>0:
        #print("------------------------4")
        col=col-1
        if(puzzle[cell[0]][row][col]=="b"):
            #print(cell, "no pass4")
            return False
        if(puzzle[cell[0]][row][col]=="#"or str(puzzle[cell[0]][row][col]).isupper() or type(puzzle[cell[0]][row][col])==int):
            break
    #print("------------------------finish")
    return True
        
        
        

def possible_pos(puzzle,empty,lit_pos,no_bulb_area):
    print("empty",empty)
    print("lit_pos",lit_pos)
    print("no_bulb_are",no_bulb_area)
    
    res=[]
    if(len(empty[0])==0):
        for pos in empty[1]:
            if(check(puzzle,[1,pos[0],pos[1]]) and [1,pos[0],pos[1]] not in no_bulb_area) :
                if(is_valid(puzzle,pos[0],pos[1],1)==True):
                    res.append([1,pos[0],pos[1]])
                    
    elif(len(empty[0])!=0 and len(empty[1])!=0):
        for index in range(len(empty)):
            for pos in empty[index]:
                if(check(puzzle,[index,pos[0],pos[1]]) and [index,pos[0],pos[1]] not in no_bulb_area):
                    if(is_valid(puzzle,pos[0],pos[1],index)==True):
                        res.append([index,pos[0],pos[1]])
    elif(len(empty[1])==0):
        for pos in empty[0]:
            if(check(puzzle,[0,pos[0],pos[1]]) and [0,pos[0],pos[1]] not in no_bulb_area):
                if(is_valid(puzzle,pos[0],pos[1],0)==True):
                     res.append([0,pos[0],pos[1]])
    if(len(lit_pos[0])==0):
        for pos in lit_pos[1]:
            if(check(puzzle,[1,pos[0],pos[1]]) and [1,pos[0],pos[1]] not in no_bulb_area) :
                if(is_valid(puzzle,pos[0],pos[1],1)==True):
                    res.append([1,pos[0],pos[1]])
    elif(len(lit_pos[0])!=0 and len(lit_pos[1])!=0):
        for index in range(len(lit_pos)):
            for pos in lit_pos[index]:
                if(check(puzzle,[index,pos[0],pos[1]]) and  [index,pos[0],pos[1]] not in no_bulb_area):
                    if(is_valid(puzzle,pos[0],pos[1],index)==True):
                        res.append([index,pos[0],pos[1]])
    elif(len(lit_pos[1])==0):
        for pos in lit_pos[0]:
            if(check(puzzle,[0,pos[0],pos[1]]) and [0,pos[0],pos[1]] not in no_bulb_area):
                if(is_valid(puzzle,pos[0],pos[1],0)==True):
                    res.append([0,pos[0],pos[1]])
    return res
                    

                
        
    

In [35]:
def light_up2(wormhole_added_grids: list, bulb_loc: list, empty_and_not_lit_loc: list):
    '''
    update record after adding a bulb on bulb_loc
    parameters:
        wormhole_added_grids: grids already add wormholes
        bulb_loc: the location that you want to put a bulb 
            eg:[x,y,z] - xth gird, yth row, zth column
        empty_and_not_lit_loc: location of empty and not lit cells
    return:
        wormhole_added_grids: grids after adding a bulb on bulb_loc
        empty_and_not_lit_loc: location of empty and not lit cells
    '''
    wormhole_added_grids[bulb_loc[0]][bulb_loc[1]][bulb_loc[2]] = 'b'
    # light up
    
    ## up
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[1] -= 1
    while cur_loc[1]>=0:
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#' or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
            break
        # case4: jump in a loop of wormhole
        if cur_loc == bulb_loc:
            break
            
        cur_loc[1] -= 1
        
    ## down
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[1] += 1
    while cur_loc[1]<len(wormhole_added_grids[cur_loc[0]]):
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#' or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
            break    
        # case4: jump in a loop of wormhole
        if cur_loc == bulb_loc:
            break
            
        cur_loc[1] += 1
        
    ## left
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[2] -= 1
    while cur_loc[2]>=0:
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#'or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
            break
        # case4: jump in a loop of wormhole
        if cur_loc == bulb_loc:
            break
            
        cur_loc[2] -= 1
        
    ## right
    cur_loc = copy.deepcopy(bulb_loc)
    cur_loc[2] += 1
    while cur_loc[2]<len(wormhole_added_grids[cur_loc[0]][0]):
        # case1: current cell is empty and not lit by other bulbs -> light up it
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '_':
            wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] ='^'
            empty_and_not_lit_loc[cur_loc[0]].remove((cur_loc[1],cur_loc[2]))
            
        # case2: current cell is a wormhole -> go to another gird's corresponding wormhole
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] in [chr(i) for i in range(65,91)]:
            wormhole = wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] # eg: 'A','B'...
            tp_grid_index = 1-cur_loc[0]
            tp_i = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) // len(wormhole_added_grids[tp_grid_index][0])
            tp_j = list(_flatten(wormhole_added_grids[tp_grid_index])).index(wormhole) % len(wormhole_added_grids[tp_grid_index][0])
            cur_loc = [tp_grid_index, tp_i, tp_j]
        # case3: current cell is a wall -> stop here
        if wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]] == '#' or type(wormhole_added_grids[cur_loc[0]][cur_loc[1]][cur_loc[2]])==int:
        # case4: jump in a loop of wormhole
            break
        if cur_loc == bulb_loc:
            break
            
        cur_loc[2] += 1
        
    return wormhole_added_grids, empty_and_not_lit_loc

In [36]:
def display(puzzle):
    for (left,right) in zip(puzzle[0], puzzle[1]):
        for i in left:         
            print(i,end="|")
        print("     ",end=" ")
        for i in right:
            print(i,end="|")
        print("")  

In [37]:
def finish_lightup(empty):
    if len(empty[0])==0 and len(empty[1])==0:
        return True
    return False
    

In [38]:
def get_wall(puzzle):
    res=[]
    for index in range(len(puzzle)):
        for i in range(len(puzzle[index])):
            for j in range((len(puzzle[index][i]))):
                if(type(puzzle[index][i][j])==int and puzzle[index][i][j]>=1):
                    res.append([index,i,j])
    return res


def get_neb(i,j,puzzle,index):
    res=[]
    if i-1>=0 and puzzle[index][i-1][j]=="^" :
        res.append([index,i-1,j])
    if j-1>=0 and puzzle[index][i][j-1]=="^":
        res.append([index,i,j-1])
    if i+1<len(puzzle[0]) and puzzle[index][i+1][j]=="^":
        res.append([index,i+1,j])
    if j+1<len(puzzle[0][0]) and puzzle[index][i][j+1]=="^" :
        res.append([index,i,j+1])
    return res
        
def recheck(puzzle,no_bulb_area):
    number_list=get_wall(puzzle)
    for cell in number_list:
        bulb_around=count_neb(cell[1],cell[2],puzzle,cell[0],"b")
        if(bulb_around<puzzle[cell[0]][cell[1]][cell[2]]):
            wall_around=get_neb(cell[1],cell[2],puzzle,cell[0])
            for cell2 in wall_around:
                if(check(puzzle,cell2) and [cell2[0],cell2[1],cell2[2]] not in no_bulb_area):
                    puzzle[cell2[0]][cell2[1]][cell2[2]]="b"
                    
                    
            
            
            
        
     
    

In [39]:
def solve(puzzle,empty,lit_pos,count,no_bulb_area):
    print("----------------------")
    if(finish_lightup(empty)==True):
        recheck(puzzle,no_bulb_area)
        print("found one solution")
        count[0]+=1
        if (count[0] <= 2):
            display(puzzle)
            if (count[0] == 2):
                raise SolutionNumberExceed
    else:
        display(puzzle)
        possible_list=possible_pos(puzzle,empty,lit_pos,no_bulb_area) 
        old_puzzle=copy.deepcopy(puzzle)
        print(possible_list)
        print("poslist",len(possible_list))
        #display(puzzle)
        if(len(possible_list)==0):
            print("no more cell can place light")
            
            
        else:
            light_up2(puzzle, possible_list[0], empty)
            empty2=find_empty(puzzle)
            print("empty",empty2)
            lit_pos2=lit_position(puzzle)
            print("lit_pos",lit_pos2)
            solve(puzzle,empty2,lit_pos2,count,no_bulb_area)
            puzzle=old_puzzle

            
  

In [40]:


def game_start(number_added_grids,walls_loc):
    puzzle=copy.deepcopy(number_added_grids) # get generated unsolved puzzle
    no_bulb_area=no_bulbs(walls_loc,puzzle) # get cells that can not place bulb
    curr_bulbs=place_bulbs(walls_loc,puzzle) # get the bulbs list that we can place it in the right cell for sure
    empty_loc=generate_empty(puzzle)   # get the cells that have not been lighted 
    wall_neb=generate_wall_neb(empty_loc,puzzle) # get the wall's neighbours
    light_step1(curr_bulbs,puzzle,empty_loc,wall_neb) # place the bulbs and update the puzzle, light the right cells
    update_empty=find_empty(puzzle) # update the unlight cell list
    already_lit_pos=lit_position(puzzle) # get the cell list that are already light
    possible_list=possible_pos(puzzle,update_empty,already_lit_pos,no_bulb_area) # get the possible cells that can be plcae the bulbs
    new_bulbs=place_bulbs(walls_loc,puzzle) # since we update the puzzle, so we can get the bulbs list that we can place it in the right cell for sure again
    new_empty=find_empty(puzzle) # update empty cell list
    for bulb in new_bulbs:
        if check(puzzle,bulb):
            light_up2(puzzle, bulb, new_empty) # light cell 
    no_bulb_area2=no_bulbs(walls_loc,puzzle)
    update_empty2=find_empty(puzzle)
    lit_pos2=lit_position(puzzle)
    possible_list2=possible_pos(puzzle,update_empty2,lit_pos2,no_bulb_area)
    solve(puzzle,update_empty2,lit_pos2,[0],no_bulb_area) # pick possible cell that can place a bulb
    return puzzle
    
    
    

In [43]:
new_puzzle=create_unverified_puzzle((6,6),0.25, True)
puzzle=game_start(new_puzzle,walls_loc)

empty [[(0, 1), (1, 0), (1, 1), (1, 2), (2, 1), (2, 3), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 2), (5, 4), (5, 5)], [(1, 0), (1, 4), (1, 5), (2, 0), (2, 3), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 3), (4, 5), (5, 4), (5, 5)]]
lit_pos [[(0, 3), (0, 4), (0, 5)], [(0, 3), (0, 4), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2)]]
no_bulb_are [[0, 1, 0], [0, 0, 1]]
empty [[(0, 1), (1, 0), (1, 1), (1, 2), (2, 1), (2, 3), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 2), (5, 4), (5, 5)], [(1, 0), (1, 4), (1, 5), (2, 0), (2, 3), (2, 4), (3, 0), (3, 1), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 3), (4, 5), (5, 4), (5, 5)]]
lit_pos [[(0, 3), (0, 4), (0, 5)], [(0, 3), (0, 4), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2)]]
no_bulb_are [[0, 1, 0], [0, 0, 1]]
----------------------
0|_|A|^|^|^|      1|1|b|^|^|A|
_|_|_|1|#|#|      _|1|^|1|_|_|
0|_|2|_|1|_| 

In [44]:
display(puzzle)

0|^|A|^|^|^|      1|1|b|^|^|A|
^|b|^|1|#|#|      b|1|^|1|b|^|
0|^|2|b|1|^|      ^|0|^|^|^|B|
^|^|b|^|^|^|      ^|^|^|b|^|^|
b|B|^|^|b|^|      ^|b|^|^|#|^|
^|1|^|2|^|b|      0|#|^|1|b|^|
